In [2]:
import os
# this code is necessary only if you have more than 1 GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import tensorflow as tf
import keras as keras
import os
from keras.layers import Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.models import load_model
import numpy as np
from pathlib import Path

import os
import shutil

Using TensorFlow backend.


ada contoh binary classification dengan flow_from_directory di sini:

https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8

https://jkjung-avt.github.io/keras-tutorial/

https://medium.com/@parthvadhadiya424/hello-world-program-in-keras-with-cnn-dog-vs-cat-classification-efc6f0da3cc5

https://www.kaggle.com/sentdex/full-classification-example-with-convnet

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


First part is splitting dataset into training set and validation set. For this demo we only use 1000 images, 800 images as training set, and 200 images as validation set. 

In [4]:
# preparing parameters     
image_dir_cat='../data/train/cat' # assuming cat & dog images has been separated in  different directories
image_dir_dog='../data/train/dog'


session = "simple1000" # to differentiate between runs
ClassNames = ['cat', 'dog']
data_dir="../simple1000" # to differentiate between runs
learning_rate = 0.0001
img_width = 331 # 331 for pre-trained nasnet
img_height = 331
nbr_epochs = 10 
batch_size = 4 # batch size depends on available memory on GPU. GTX 1080 Ti use (4)
np.random.seed(2018)
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"    
number_of_class=len(ClassNames)
print("train directory : ", train_dir)
print("valid directory : ", valid_dir)    
print("number of classes: "+ str(number_of_class))
  
logfile = session + '-train' + '.log'
print("logfile   :", logfile)    

train directory :  ../simple1000/train
valid directory :  ../simple1000/valid
number of classes: 2
logfile   : simple1000-train.log


In [5]:
# make training directory
# make validation directory
# copy images to respective directories
print("copy start")        
def MakeDir(newdir):
    if not os.path.exists(newdir):
        os.makedirs(newdir)

        # make validation & training directories, if not exist yet        
MakeDir(valid_dir)
MakeDir(valid_dir+'/cat')
MakeDir(valid_dir+'/dog')
MakeDir(train_dir)
MakeDir(train_dir+'/cat')
MakeDir(train_dir+'/dog')

# copy files to working directories
print("copy cats")
counter=0
for root, dirs, files in os.walk(image_dir_cat):
    for file in files:
        fullfilename = os.path.join(root, file)
#        print(str(counter) + ": " + fullfilename)
        if(counter<800):
            shutil.copyfile(fullfilename,train_dir+"/cat/"+file)            
        if(counter>=800 and counter<1000):
            shutil.copyfile(fullfilename,valid_dir+"/cat/"+file)
        if(counter>=1000):
            break
        counter=counter+1                       
print("copy dogs")            
counter=0        
for root, dirs, files in os.walk(image_dir_dog):
    for file in files:
        fullfilename = os.path.join(root, file)
#        print(str(counter) + ": " + fullfilename)
        if(counter<800):
            shutil.copyfile(fullfilename,train_dir+"/dog/"+file)            
        if(counter>=800 and counter<1000):
            shutil.copyfile(fullfilename,valid_dir+"/dog/"+file)
        if(counter>=1000):
            break
        counter=counter+1
print("copy finished")        

copy start
copy cats
copy dogs
copy finished


In [6]:
# make model with transfer learning
if(True):
    model_notop = keras.applications.nasnet.NASNetLarge(input_shape=(img_width, img_height, 3),
                                                                  include_top=False,
                                                                  weights='imagenet', input_tensor=None,
                                                                  pooling=None)
        # add a global spatial average pooling layer
    x = model_notop.output
    x = GlobalAveragePooling2D()(x)        
    x = Dense(1024, activation='relu')(x) # let's add a fully-connected layer        
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='sigmoid')(x)
    deep_model = Model(model_notop.input, predictions)

In [7]:
#  training
if(True):
    sgd_optimizer = SGD(lr=learning_rate, momentum=0.9, decay=0.0, nesterov=True)
    deep_model.compile(loss='binary_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

    # set up callbacks
    
    csv_logger = CSVLogger(logfile, append=True)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')
    best_model_file=session+'-weights.{epoch:02d}-{val_loss:.2f}.h5'
    # best_model_file = session + '-weights' + '.h5'
    best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose=1, save_best_only=True)

    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

    val_datagen = ImageDataGenerator(rescale=1. / 255)

    print('prepare train generator')
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,        
        class_mode='binary')
    print('prepare validation generator')
    validation_generator = val_datagen.flow_from_directory(
        valid_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='binary')
    print('fit generator')
    deep_model.fit_generator(
        generator=train_generator,
        #        steps_per_epoch=nbr_train_samples/batch_size, # in Keras 2.2.0, automatically acquired from train generator
        epochs=nbr_epochs,
        verbose=1,
        validation_data=validation_generator,
        #        validation_steps=nbr_validation_samples/batch_size, # automatically acquired from validation generator
        callbacks=[best_model, csv_logger, early_stopping])

prepare train generator
Found 1600 images belonging to 2 classes.
prepare validation generator
Found 400 images belonging to 2 classes.
fit generator
Epoch 1/10
400/400 [==============================] - 271s 677ms/step - loss: 0.3706 - acc: 0.8287 - val_loss: 0.1626 - val_acc: 0.9625

Epoch 00001: val_acc improved from -inf to 0.96250, saving model to simple1000-weights.01-0.16.h5
Epoch 2/10
400/400 [==============================] - 222s 556ms/step - loss: 0.3033 - acc: 0.8769 - val_loss: 0.1101 - val_acc: 0.9700

Epoch 00002: val_acc improved from 0.96250 to 0.97000, saving model to simple1000-weights.02-0.11.h5
Epoch 3/10
400/400 [==============================] - 223s 557ms/step - loss: 0.2929 - acc: 0.8731 - val_loss: 0.1433 - val_acc: 0.9625

Epoch 00003: val_acc did not improve from 0.97000
Epoch 4/10
400/400 [==============================] - 223s 556ms/step - loss: 0.2903 - acc: 0.8931 - val_loss: 0.1143 - val_acc: 0.9800

Epoch 00004: val_acc improved from 0.97000 to 0.98000

In [8]:
#prediction
nbr_test_samples=12500 

#choose weights file manually
weights_path = 'simple1000-weights.07-0.09.h5'
test_data_dir = '../data/test/'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = False, # no shuffling, since filenames must match predictions. Shuffling may change file sequence
        classes = None, # 
        class_mode = None)

test_image_list = test_generator.filenames
print('Loading model and weights')
predict_model = load_model(weights_path)

print('Begin to predict for testing data ...')
predictions = predict_model.predict_generator(test_generator, nbr_test_samples)

np.savetxt(session+'-predictions.txt', predictions) # store prediction matrix, for later analysis if necessary

Found 12500 images belonging to 1 classes.
Loading model and weights
Begin to predict for testing data ...


In [10]:
# submission
submission_file=session+'-submit.csv'
print('Begin to write submission file:'+submission_file)

f_submit = open(submission_file, 'w')
f_submit.write('id,label\n')
for i, image_name in enumerate(test_image_list):
    basename=os.path.basename(image_name)
    filename, fileext = os.path.splitext(basename)    
    prediction_class    =predictions[i][0] # get predictions from array        
    f_submit.write('%s,%s\n' % (filename, prediction_class))
f_submit.close()
print('Finished write submission file ..')

Begin to write submission file:simple1000-submit.csv
Finished write submission file ..
